## Deploy your pre-trained keras model to AWS

In [1]:
pip install tensorflow==2.6

     |████████████████████████████████| 458.3 MB 9.1 kB/s             
     |████████████████████████████████| 42 kB 2.1 MB/s             
     |████████████████████████████████| 4.0 MB 57.1 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 5.8 MB 48.7 MB/s            
     |████████████████████████████████| 1.6 MB 63.6 MB/s            
     |████████████████████████████████| 462 kB 60.2 MB/s            
     |████████████████████████████████| 132 kB 74.3 MB/s            
     |████████████████████████████████| 781 kB 64.1 MB/s            
     |████████████████████████████████| 4.9 MB 40.0 MB/s            
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30702 sha256=a4d06ec235f81705d35f58eeb0db994dbe5357689c2dd267d994fe5d64e98b29
  Stored in directory: /home/ec2-user/.cache/pip/wheels/22/4c/94/0583f60c9c5b6024ed64f290cb2d43b06bb4f75577dc3c93a7
Successfully built clang
  Attempting uninstall: typing-extensions
    Foun

In [ ]:
pip install keras==2.6

In [2]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


In [34]:
import numpy as np

# if your model is saved as only a .h5 file
MODEL_LOCATION ='sampleModel'


## 2. Load Your Model
Simply run the cell below; the model will be loaded based on how you defined the above

In [35]:
if MODEL_LOCATION!='': #if your model is saved as a .h5 file only
    from keras.preprocessing.image import img_to_array
    from keras.models import load_model
    model = load_model(MODEL_LOCATION, compile = False)#ad the model
    print("loaded model from MODEL_LOCATION")

loaded model from MODEL_LOCATION


In [36]:
import tensorflow as tf
tf.__version__

'2.6.0'

## 3. Convert the Keras Model to the format AWS wants
- Converts to a Protobuff file
- Saves it in a certain aws file structure
- Tarballs this file and zips it

In [37]:
def convert_to_aws(loaded_model):
    """
    given a pre-trained keras model, this function converts it to a TF protobuf format
    and saves it in the file structure which aws expects
    """  
    from tensorflow.python.saved_model import builder
    from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
    from tensorflow.python.saved_model import tag_constants
    import tensorflow as tf
    import os
    import shutil
    if tf.executing_eagerly():
       tf.compat.v1.disable_eager_execution()
    dirpath = 'export'
    if os.path.exists(dirpath) and os.path.isdir(dirpath):
        shutil.rmtree(dirpath)
    
    # This is the file structure which AWS expects. Cannot be changed. 
    model_version = '1'
    export_dir = 'export/Servo/' + model_version
    
    # Build the Protocol Buffer SavedModel at 'export_dir'
    builder = builder.SavedModelBuilder(export_dir)
    
    # Create prediction signature to be used by TensorFlow Serving Predict API
    signature = predict_signature_def(
        inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})
    
    session = tf.compat.v1.Session()
    init_op = tf.compat.v1.global_variables_initializer()
    session.run(init_op)
    # Save the meta graph and variables
    builder.add_meta_graph_and_variables(
        sess=session, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save() 
    
    #create a tarball/tar file and zip it
    import tarfile
    with tarfile.open('model_deploy.tar.gz', mode='w:gz') as archive:
        archive.add('export', recursive=True)
        
convert_to_aws(model)

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/Servo/1/saved_model.pb


## 4. Move the tarball (tar.gz) to S3

In [24]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model_deploy.tar.gz', key_prefix='model')

# View details of the uploaded bucket
print(f"Bucket name is: {sagemaker_session.default_bucket()}")

Bucket name is: sagemaker-us-east-1-852069333125


This is the name of the bucket which SageMaker made in S3

In [25]:
# where did it upload to?
print("Bucket name is:")
sagemaker_session.default_bucket()

Bucket name is:


'sagemaker-us-east-1-852069333125'

## 5. Create a SageMaker Model
First, create an empty train.py file (TensorFlowModel expects this at its 'entry point', but can be empty)

In [26]:
!touch train.py #create an empty python file

In [27]:
import boto3, re
import tensorflow as tf
from sagemaker import get_execution_role

# the (default) IAM role you created when creating this notebook
role = get_execution_role()

# Create a Sagemaker model (see AWS console>SageMaker>Models)
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model_deploy.tar.gz',
                                  role = role,
                                  framework_version = tf.__version__,
                                  entry_point = 'train.py')

## 6a) Host the SageMaker model and
## 6b) Create an Endpoint to access the model 

Ignore the message `update_endpoint is a no-op in sagemaker>=2`

In [28]:
deployement_instance_type = "ml.m4.xlarge"
from sagemaker.serializers import CSVSerializer
predictor = sagemaker_model.deploy(initial_instance_count=1, instance_type=deployement_instance_type)
endpoint = predictor.endpoint

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


---------!

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [29]:
endpoint

'tensorflow-inference-2022-05-21-06-57-20-604'

## Success! You have deployed a keras model into AWS
# ---------------------
### 7. Confirm its working correctly by making a prediction
Now, we want to use our endpoint/model. Create a predictor which uses the endpoint

In [38]:
import cv2
import mediapipe as mp
import numpy as np
import copy
import itertools

In [39]:
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []
    
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        
        landmark_point.append([landmark_x, landmark_y])

    return landmark_point


def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    
    temp_landmark_list = list(itertools.chain.from_iterable(temp_landmark_list))

    
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

def get_key(val):
    for key, value in labels_dict.items():
        if val == value:
            return key
    return "key doesn't exist"

In [40]:

mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils
labels_dict = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'J':9,'K':10,'L':11,'M':12,'N':13,'O':14,'P':15,'Q':16,'R':17,'S':18,'T':19,'U':20,'V':21,'W':22,'X':23,'Y':24,'Z':25,'space':26,'del':27,'nothing':28}

In [46]:
jsonData["predictions"]

[[0.0344865844,
  0.0345011503,
  0.0345036052,
  0.0344779491,
  0.0344655961,
  0.0344716683,
  0.0344790183,
  0.0344779193,
  0.0344862677,
  0.0344822742,
  0.0344918594,
  0.0344807506,
  0.0344863646,
  0.0344770104,
  0.0344935954,
  0.0344843082,
  0.0344746634,
  0.0344893746,
  0.0344714597,
  0.0344845653,
  0.0344930626,
  0.0344822407,
  0.0344895683,
  0.0344705805,
  0.0344851501,
  0.0344789103,
  0.0344848745,
  0.0344713852,
  0.0344783515]]

In [48]:
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker.serializers import CSVSerializer
import json


endpoint = 'tensorflow-inference-2022-05-21-06-57-20-604' #get endpoint name from SageMaker > endpoints

predictor=sagemaker.predictor.Predictor(
    endpoint_name=endpoint,
    sagemaker_session=sagemaker.Session(),
    serializer=CSVSerializer())

#success, image = cap.read()
image = cv2.imread("W_test.jpg", cv2.IMREAD_COLOR)
imgRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
results = hands.process(imgRGB)
predictions=""
if results.multi_hand_landmarks:
    for hand_landmarks, handedness in zip(results.multi_hand_landmarks,results.multi_handedness):
        landmark_list = calc_landmark_list(imgRGB, hand_landmarks)
        pre_processed_landmark_list = pre_process_landmark(landmark_list)
        data = np.array([pre_processed_landmark_list], dtype=np.float32)
        predictions = predictor.predict(data)
        my_json = predictions.decode("utf-8").replace("'", '"')
        jsonData = json.loads(my_json)
        classes_x=np.argmax(np.squeeze(jsonData["predictions"]))
        print("\nLabel")
        print(get_key(classes_x))
        print("\n")
        cv2.putText(image, get_key(classes_x),(10,60), cv2.FONT_HERSHEY_PLAIN,3, (255,0,255),4)




Label
C




# Access SageMaker endpoint using lamda function and Api gateway

In [47]:
import requests
URL = 'https://pgjw2grj50.execute-api.us-east-1.amazonaws.com/test/predict'
header = {"Content-Type":"application/json"}
data_payload = '{"data":"0.0,0.0,0.14117648,-0.01764706,0.2,-0.1882353,0.05882353,-0.30588236,-0.0882353,-0.3764706,0.27058825,-0.44117647,0.3647059,-0.63529414,0.41764706,-0.7764706,0.44117647,-0.9,0.14117648,-0.4882353,0.1882353,-0.7176471,0.20588236,-0.87058824,0.21176471,-1.0,0.02352941,-0.46470588,-0.01764706,-0.67058825,-0.04117647,-0.81764704,-0.05882353,-0.9588235,-0.0882353,-0.3882353,-0.0882353,-0.42941177,-0.07058824,-0.32941177,-0.06470589,-0.24705882"}'
request = requests.request("POST",URL, headers=header, data=data_payload)
print(request.text)

[[0.0344889313, 0.0345029645, 0.0345036276, 0.0344779342, 0.0344665162, 0.0344712362, 0.0344801955, 0.034477856, 0.0344871096, 0.0344826579, 0.0344917662, 0.034480419, 0.0344854817, 0.0344761573, 0.0344930738, 0.0344839618, 0.0344761759, 0.0344885178, 0.0344721, 0.0344843306, 0.0344948731, 0.0344801247, 0.034489, 0.0344701111, 0.0344847441, 0.0344773047, 0.034485057, 0.0344713032, 0.0344764814]]


## Cleanup!

else you will incur extra charges

https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html

- Stop Notebook
- delete endpoints
- delete models
- delete S3 bucket
- delete cloudwatch groups
